In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

In [1]:
import sqlite3
import pandas as pd
sqlite3.sqlite_version

'3.38.0'

In [2]:
conn = sqlite3.connect('C:\sql_project\mydb.db') 

In [3]:
crm = pd.read_csv('/content/drive/MyDrive/Python/GB Четвертная работа/crm_data.csv') 
visits = pd.read_csv('/content/drive/MyDrive/Python/GB Четвертная работа/site_visits_data.csv') 
crm['application_ts'] = pd.to_datetime(crm['application_ts'].apply(lambda x: x.split(' UTC')[0]))
visits['visit_start_time'] = pd.to_datetime(visits['visit_start_time'].apply(lambda x: x.split(' UTC')[0]))
crm.to_sql('crm', conn, if_exists='replace', index=False)
visits.to_sql('visits', conn, if_exists='replace', index=False)

In [4]:
def fetch_query(query):
    return pd.read_sql_query(query, conn)

In [5]:
query = """
SELECT * 
FROM crm
LIMIT 3
"""
fetch_query(query)

,user_id,id_contact,age,spoke_with_agent,application_ts,documents_send,documents_recieved,flag
0,0947abecd9b9ee653139dcafaf48e243,718951688,28,1,2019-07-19 16:07:25,0,0,0
1,baaa6faf65e6a37630e43520d061a0aa,721632137,29,1,2019-07-22 16:02:23,0,0,0
2,856d1cf0187a188a40f9749fdd28cc3d,718659195,20,0,2019-07-19 07:09:59,1,0,0


In [6]:
query = """
SELECT * 
FROM visits
LIMIT 3
"""
fetch_query(query)

,user_id,visit_start_time,session_duration,delivery_pickup,bounce_rate,total_sessions,user_type,visited_partners,step_1,step_2,step_3
0,1fb42a882f576c05f226ad3bf313caa1,2019-07-22 06:12:57,0,not_stated,1,1,new,0,0,0,0
1,7b2eb4009b25b2b02589b8b60fd8a3e3,2019-07-22 08:04:27,0,not_stated,1,1,old,0,0,0,0
2,39f4448e01e3126a9b35b32ae5b36bbe,2019-07-22 13:39:54,0,not_stated,1,1,old,0,0,0,0


In [7]:
#1) Написать запрос, который считает количество пользователей, разговаривавших с агентом 20-го и 21-го числа

query = """
SELECT COUNT(user_id) 
FROM crm
WHERE spoke_with_agent = 1 AND strftime('%d', application_ts) IN ('20','21')
"""
fetch_query(query)

,COUNT(user_id)
0,1953


In [8]:
#2) Написать запрос, который отвечает на следующий вопрос: “Есть ли пользователи,которые перескакивали шаги в воронке 
# (т.е. например они прошли шаг3, но не проходили шаг 2. Если да, то сколько их?

query = """
SELECT user_id, step_1, step_2, step_3 
FROM visits
WHERE step_2 > step_1 OR step_3 > step_2 OR step_3 > step_1
LIMIT 10
"""
fetch_query(query)

,user_id,step_1,step_2,step_3
0,0d828cbe3b0cace3911cee67bbb2cefa,0,0,1
1,244ef04b2f89c05a1bb2e02531d2620f,1,0,1
2,ca8c13574141e1b84ef73673fe972117,0,0,1
3,3861d49bb9c58aff79b4f65f295f1cf1,0,0,1
4,a24cb91feae84fb318faaaf54b752d13,0,0,1
5,1e0deddf03d7f90903b6590e5e5a4b6e,0,0,1
6,8f232ec53c2f7b63d96da0fd5b0c9fff,0,0,1
7,0dcbf54dfc83660787052fc10a219b2b,0,0,1
8,60f7db4fadb5fdf2a06277c5f9b840e1,0,0,1
9,3b410ee0e21e1135cd161d84cb2863ef,0,0,1


In [9]:
query = """
SELECT COUNT(user_id)
FROM visits
WHERE step_2 > step_1 OR step_3 > step_2 OR step_3 > step_1
"""
fetch_query(query)

,COUNT(user_id)
0,2791


In [10]:
#3) Может ли пользователь иметь 2 статуса, т.е. быть старым и новым? Напишите запрос, отвечающий на этот вопрос

query = """
SELECT user_id, COUNT(DISTINCT user_type)
FROM visits
GROUP BY user_id
HAVING COUNT(DISTINCT user_type) = 2
"""
fetch_query(query)

,user_id,COUNT(DISTINCT user_type)
0,000e29dd39c4030f4f69ca9b3497ac8c,2
1,0023d62e079ee14d5488c29db03ccb38,2
2,003673b3fda12cb05d2aa8467c12d6cc,2
3,0044fbae9b62c0c737e407bde4776e99,2
4,00548d83f78535d396a107951b6ebaf2,2
...,...,...
8516,ffc0456995a857931d9bd725a4062b8c,2
8517,ffd6d668ad8a3a83b36f0a59b346073e,2
8518,ffdb3f9f139395b52b7f180dd867cfac,2
8519,ffe0a99cb9c5d08c9e4b4df26270d8dc,2


In [11]:
#4) Посчитайте среднее время, которое проходит между визитом на сайт и подачей заявки (application_ts и visit_start_time соотв.)

query = """
SELECT AVG(TIME(application_ts) - TIME(visit_start_time)) AS avg_time_difference
FROM crm c INNER JOIN visits v ON c.user_id = v.user_id
"""
fetch_query(query)

,avg_time_difference
0,0.053329


In [12]:
#5) Может ли пользователь иметь 2 или больше различных типа доставки? Напишите запрос, отвечающий на этот вопрос

query = """
SELECT user_id, COUNT(DISTINCT delivery_pickup)
FROM visits
GROUP BY user_id
HAVING COUNT(DISTINCT delivery_pickup) >= 2
"""
fetch_query(query)

,user_id,COUNT(DISTINCT delivery_pickup)
0,000e29dd39c4030f4f69ca9b3497ac8c,2
1,0023d62e079ee14d5488c29db03ccb38,2
2,0054a9477c1caa935b436b9572c28cbb,2
3,0058ffbb0a0af76d7f9910c698366ae6,2
4,00675014637f17707f82f865c9660b0b,2
...,...,...
2642,ff2028137e3d2889311724e29fd3bb40,2
2643,ff4140e398e5066854a7519fb82113d7,2
2644,ff70a3db3f2449cb783a54e64068e94c,2
2645,ff7f9558eb51ef16bd9b2b5f005c886a,2


In [13]:
#6*) Напишите запрос, который присваивает порядковый номер сессии для всех сессий конкретного пользователя (оконная функция)

query = """
SELECT user_id, visit_start_time, ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY visit_start_time) AS row_number
FROM visits
"""

fetch_query(query)

,user_id,visit_start_time,row_number
0,000a572cb1073e39b7673d10afb4bd62,2019-07-19 21:10:31,1
1,000e29dd39c4030f4f69ca9b3497ac8c,2019-07-21 15:54:47,1
2,000e29dd39c4030f4f69ca9b3497ac8c,2019-07-23 10:02:04,2
3,0010123f4379a11a2aa7ea0cedecbcda,2019-07-20 18:53:28,1
4,00132fb0c7096ec2636110ae8bb5b9a7,2019-07-21 15:18:27,1
...,...,...,...
43396,fff6e87042bf7e39632c256cede4f847,2019-07-22 10:10:44,1
43397,fff73d7e15c9929ef6cd82bd19f9ab1e,2019-07-22 06:51:54,1
43398,fff9a7934a7491903b326cc4d9b096cd,2019-07-19 16:21:14,1
43399,fffa2c81dbb6aed74420d737bb73c4b9,2019-07-21 12:34:33,1
